# -Introduction
l'agence de stante publique France a lance un apel a projets pour trouver
des idees innovantes d'applicationq en lien avec l'alimentation
Nous avons decidé de nous attaquer a l'obésite.
Nous voulons etudier la possiblite qu'une application informe, pour n'importe quel produit,
le consommation du caractere Nutriscore.
La prediction du Nutriscore doit etre realise a partir de la liste des nutiments du produit.
Le Nutriscore est une classification des produits alimentaires, les classes sont nommées par ordre
de qualite nutritionnelle de A a E


## Hypothese principal :
Est il possibel de predire le Nutriscore a partire de la declaration nutitionnelle obligatoire du produit?
La declaration nutritionnelle suivant le reglement INCO

## Hypothese secondaires :
Pouvons nous classer nos produits en 3 categories: {divers,matieresgrasses ajoutees, boissons}
a partir de la declaration nutritionnelle obligatoire du produit? a minima a a partir d'une liste etendue de nutriments?

## demarche de resolution:
Nous allons devoir resoudre un problem de classification, et donc pour repiondre a ces questions, nous devons analyser notre jeu de donnes pour determiner si les nutriments agissent sur la classe du nutriscore.
Pour cela nous allons devoir nous concentrer sur les caracteristiques essentiels:

#### Nom des colonnes = [code_barres, nutriscore_grade, energie_100g, matieres_grasses_100g, matieres_grasses_saturees_100g, glucides_100g, glucides_sucres_100g, protein_100g, sel_100g]
# jeu de donnees:
Open Food FACTS.
    * les information generales sur la fiche du produit: nom, date de modification, etc.
    * un ensemble de tags: categorie du produit, localisation, origine, etc.
    * les ingredients composant les produits et leurs additifs eventuels.
    * des informatrion nutritionnelles: quantite en grammes d'un nutriment pour 100 grammes du produit.

# Librairies : 
    * Numpy permet de manipuler des tableaux de nombres uniquement ainsi que des fonctions mathematiqueq operant sur ces tableaux
    * pandas etend les possiblites de numpy en permettant de manipuler facilement des tableaux de données mixtes.
    * seaborn permet de realiser des graphique et des statistique sir des dataframes de pandas.

# Fonctions Helper:
Nous allons utiliser ces fonctions d'aides afin de reduire le temps de nettoyage et analyse:
    * percentage_nan_by_columns().
    * get_colors_from_listedcolormap().
    * get_set_of_labels().
    * plot_sghisto().
    * do_krustal_test().

# Collection des donnees :
##### Quelle source de donees existe-t-il?
il est demande expressement d'utiliser les donnees provenant d'open food Facts.
les champs sont separers en quatre section:
    * Les informations generales sur la fiche du produit: nom, date de modification, etc?
    * Un ensemble de tags: categories du produit, localisation, origine, etc.
    * Les ingredients composant les produits et leur additifs eventuels
    * Les informations nutritionnelle: quantite en grammes d'un nutriment pour 100 grammes du produit.
##### Quelle est la reglementation sur la protection de ces donnees?
aucaun , la source de donnees est publique.
##### ou sont stockees les donnees?
elles sont sur les serveurs distants publiques et sur notre ordinateur en sdd local.
##### Quel est le type des donnees :
structure




# Importation des données 

In [1]:
import dask.dataframe as dd
import numpy as np
import pandas as pd
import re
import IPython.display
import matplotlib.pyplot as plt 
import seaborn as sns
plt.style.use("seaborn")
%matplotlib inline
dtypes = {
    "brand_owner":object,
    "generic_name":object,
    "ingredients_from_palm_oil_tags":object,
    "ingredients_that_may_be_from_palm_oil_tags":object,
    "origins":object,
    "origins_en":object,
    "origins_tags":object,
    "packaging_text":object,
    "traces":object,
    "traces_en":object,
    "traces_tags":object,
    "code":object,
    "abbreviated_product_name": object
    }
#data = dd.read_csv("data.csv", sep = "\t", dtype=dtypes, low_memory=False).compute()


In [4]:
url = r'https://raw.githubusercontent.com/Loridan/openclassrooms-iml-projects/main/project2/en.openfoodfacts.org.products-df5.csv'
dataframe = pd.read_csv(url,sep='\t')

In [5]:
## Afficher row of data from url
dataframe.head()

,"code_barres,date_creation,nom_produit,categories,countries,nutriscore_score,nutriscore_grade,pnns_groupes_1,pnns_groupes_2,energie_100g,matieres_grasses_100g,matieres_grasses_saturees_100g,glucides_100g,glucides_sucres_100g,fibres_100g,proteines_100g,sel_100g,sodium_100g"
0,"0000000000100,1970-01-01 00:00:01.444572561, m..."
1,"0000000001281,1970-01-01 00:00:01.517830801,Ta..."
2,"0000000001885,1970-01-01 00:00:01.511180337,Co..."
3,"0000000005470,1970-01-01 00:00:01.511778860,BA..."
4,"0000000043595,1970-01-01 00:00:01.489050424,Cr..."


Le chargment des donness retourne un evertissement sur la presence de tye different dans la colonne 0
le code a barre est realite une chaine de caractere, car certains sont trop longs pour etre stoches en type dloat 64 par le syteme


# Import des fonctions

In [6]:
# Retourne une serie comportant le pourcentage de valeurs manquantes pour chaques conlonnes du plateforme

def percentage_nan_by_columns(df):
    s = df.isnull().sum(axis=0) / dataframe.shap[0] * 100
    s = s.transform(lambda x:'{:02.2f}'.format(x) + '%')
    return s


In [8]:
# prendre connaissance du veritable contenu et de la qualite de mes donnese.
dataframe.info(verbose= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326136 entries, 0 to 326135
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                                                                           Non-Null Count   Dtype 
---  ------                                                                                                                                                                                                                                                                           --------------   ----- 
 0   code_barres,date_creation,nom_produit,categories,countries,nutriscore_score,nutriscore_grade,pnns_groupes_1,pnns_groupes_2,energie_100g,matieres_grasses_100g,matieres_grasses_saturees_100g,glucides_100g,glucides_sucres_100g,fibres_100g,proteines_100g,sel_100g,sodium_100g  326136 n

KeyError: 0